In [18]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
!pip install langchain-ollama langchain-community==0.3.25 langchain-core>=0.3.65


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# ConversationBufferMemory

In [50]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
# Clear the chat history
memory.clear()

In [51]:
memory.save_context({"input": "Hi, my name is subbu"}, {"output": "Hello subbu, how can I help?"})

In [52]:
history_string = memory.buffer
print(history_string)

Human: Hi, my name is subbu
AI: Hello subbu, how can I help?


In [53]:
# Access the list of message objects
history_messages = memory.chat_memory.messages

# You can iterate through them:
for message in history_messages:
    print(f"{type(message).__name__}: {message.content}")

HumanMessage: Hi, my name is subbu
AIMessage: Hello subbu, how can I help?


# ConversationChain

In [54]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [55]:
#from langchain_ollama import ChatOllama
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

In [56]:
llm = ChatOllama(
    model="llama3.2",      # any installed ollama model
    temperature=0.2
)

In [57]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [58]:
# Clear the chat history
conversation.memory.clear()

In [59]:
print(conversation.predict(input="Explain about Artificial Intelligence in brief."))
print(conversation.predict(input="Explain about Machine Learning in brief."))
print(conversation.predict(input="Explain about Deep Learning in brief."))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Explain about Artificial Intelligence in brief.
AI:

> Finished chain.
I'd be delighted to provide you with an overview of Artificial Intelligence (AI). AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, and making decisions.

There are several types of AI, including:

1. **Narrow or Weak AI**: This type of AI is designed to perform a specific task, such as facial recognition, speech recognition, or playing chess.
2. **General or Strong AI**: This type of AI aims to create a computer system that possesses the ability to understand, learn, 

In [60]:
chat_memory = conversation.memory

In [61]:
# Access the raw string buffer
history_string = chat_memory.buffer
print(history_string)

Human: Explain about Artificial Intelligence in brief.
AI: I'd be delighted to provide you with an overview of Artificial Intelligence (AI). AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, and making decisions.

There are several types of AI, including:

1. **Narrow or Weak AI**: This type of AI is designed to perform a specific task, such as facial recognition, speech recognition, or playing chess.
2. **General or Strong AI**: This type of AI aims to create a computer system that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks, similar to human intelligence.
3. **Superintelligence**: This hypothetical type of AI is significantly more intelligent than the best human minds, potentially leading to exponential growth in technological advancements.

AI has numerous applications in various fields, including:

1. **Virtual

In [62]:
# Access the list of message objects
history_messages = chat_memory.chat_memory.messages

# You can iterate through them:
for message in history_messages:
    print(f"{type(message).__name__}: {message.content}")

HumanMessage: Explain about Artificial Intelligence in brief.
AIMessage: I'd be delighted to provide you with an overview of Artificial Intelligence (AI). AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, and making decisions.

There are several types of AI, including:

1. **Narrow or Weak AI**: This type of AI is designed to perform a specific task, such as facial recognition, speech recognition, or playing chess.
2. **General or Strong AI**: This type of AI aims to create a computer system that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks, similar to human intelligence.
3. **Superintelligence**: This hypothetical type of AI is significantly more intelligent than the best human minds, potentially leading to exponential growth in technological advancements.

AI has numerous applications in various fields, including:

# RunnableWithMessageHistory

In [64]:
#from langchain_classic.chains import ConversationChain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import FileChatMessageHistory
#memory = ConversationBufferMemory()
#chain = ConversationChain(llm=llm, verbose=True)

In [65]:
# 1. Define your base chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"), # Placeholder for the history
    ("human", "{input}")
])

In [66]:
# 2. Define a function to get the session history (e.g., using a file)
def get_session_history(session_id: str) -> FileChatMessageHistory:
    # This function should connect to your chosen storage (DB, file, etc.)
    return FileChatMessageHistory(f"messages_{session_id}.json")

In [67]:
# Function to clear the history
def clear_chat_history(session_id: str):
    """Retrieves the history for a given session ID and clears its messages."""
    history = get_session_history(session_id)
    history.clear()
    print(f"Chat history for session '{session_id}' cleared.")
    # Optional: if using an in-memory store like the 'store' dictionary, you might also want to delete the key
    # del store[session_id] 

In [68]:
chain = prompt | llm

In [69]:
# 3. Wrap your chain with RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    clear_chat_history=clear_chat_history,
    input_messages_key="input", # Key for new input messages
    history_messages_key="chat_history" # Key for the history in the prompt template
)

In [70]:
# 4. Invoke the chain to populate the history
session_id = "my-unique-session-id"
session_config = {"configurable": {"session_id": session_id}}

In [71]:
# Clear the chat history before starting a new session
clear_chat_history(session_id) 

Chat history for session 'my-unique-session-id' cleared.


In [72]:
response1 = with_message_history.invoke(
    {"input": "Explain about Artificial Intelligence."},
    config= session_config
)

In [73]:
response2 = with_message_history.invoke(
    {"input": "Explain about Indian Economic."},
    config= session_config
)

In [74]:
response3 = with_message_history.invoke(
    {"input": "Explain about Indian History."},
    config= session_config
)

In [75]:
# 5. Retrieve the chat history
chat_history_instance = get_session_history(session_id)
messages = chat_history_instance.messages

In [78]:
# Print the messages
print(f"Chat history for session '{session_id}':")
print("\n")
for message in messages:
    print(f"- {message.type.capitalize()}: {message.content}")
    print("=============================================")

Chat history for session 'my-unique-session-id':


- Human: Explain about Artificial Intelligence.
- Ai: Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. **Learning**: AI systems can learn from data and improve their performance over time.
2. **Problem-solving**: AI systems can solve complex problems by analyzing data and making decisions.
3. **Perception**: AI systems can interpret and understand data from sensors, such as images and speech.
4. **Reasoning**: AI systems can draw conclusions and make decisions based on the data they have been trained on.

There are several types of AI, including:

1. **Narrow or Weak AI**: This type of AI is designed to perform a specific task, such as facial recognition or language translation.
2. **General or Strong AI**: This type of AI is designed to perform any intellectual task that a human can, and is still a subject of ongoing research and d